In [22]:
#pip install pysal
#pip install osmnx
#pip install spreg

In [23]:
from scipy import stats
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import osmnx as ox
sns.set(style="whitegrid")
from pysal.model import spreg
from pysal.lib import weights

In [26]:
data = pd.read_csv("data/homicidetypebyprovince.csv")
data.columns

Index(['prov_name', 'prov_istat_code_num', 'prov_acr', 'region_name',
       'reg_istat_code', 'reg_istat_code_num', 'prov_istat_code', 'geometry',
       'Year', 'region_id', 'region_name.1', 'capital_id', 'capital_name',
       'type', 'n_commited_crime', 'Year.1', 'prov_name.1', 'Type of Homicide',
       'Number of Homicide'],
      dtype='object')

In [25]:
data.head()

,prov_name,prov_istat_code_num,prov_acr,region_name,reg_istat_code,reg_istat_code_num,prov_istat_code,geometry,Year,region_id,region_name.1,capital_id,capital_name,type,n_commited_crime,Year.1,prov_name.1,Type of Homicide,Number of Homicide
0,Torino,1,TO,Piemonte,1,1,1,"POLYGON ((7.893969937235709 45.58221680371771,...",2017,15,Sicilia,15a,Agrigento,01.0 ATTENTATI,0,2006,Torino,mass murder,0
1,Vercelli,2,VC,Piemonte,1,1,2,"POLYGON ((7.929003239798809 45.74244267625574,...",2017,15,Sicilia,15a,Agrigento,02. 0 STRAGE,1,2006,Genova,mass murder,2
2,Novara,3,NO,Piemonte,1,1,3,POLYGON ((8.420786872922113 45.829807305539404...,2017,15,Sicilia,15a,Agrigento,03.0 OMICIDI VOLONTARI CONSUMATI,2,2006,Milano,mass murder,0
3,Cuneo,4,CN,Piemonte,1,1,4,MULTIPOLYGON (((6.94540332317303 44.4279439457...,2017,15,Sicilia,15a,Agrigento,03.a Omicidi a scopo di furto o rapina,0,2006,Verona,mass murder,0
4,Asti,5,AT,Piemonte,1,1,5,"POLYGON ((7.966847433785602 45.11667127823421,...",2017,15,Sicilia,15a,Agrigento,03.b Omicidio di tipo mafioso,1,2006,Venezia,mass murder,0


KeyError: 'longitude'

In [6]:
# Read OSM data - get administrative boundaries

# define the place query
query = {'city': 'Austin'}

# get the boundaries of the place (add additional buffer around the query)
boundaries = ox.geocode_to_gdf(query, buffer_dist=5000)

# Let's check the boundaries on a map
boundaries.explore()

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.